# Dual CRISPR Screen Analysis Pipeline
# Module 2
Amanda Birmingham, CCBB, UCSD (abirmingham@ucsd.edu)


<a name="table-of-contents"></a>

## Table of Contents

* [Review Goals](#review-goals)
* [Set Up Environment](#set-up-environment)
    * [Create Virtual Environment and Install Software](#create-virtual-environment-and-install-software)
    * [Initialize Shared Variables and Functions](#initialize-shared-variables-and-functions)
* [Normalize Raw Counts](#normalize-raw-counts)
    * [Set Up Constants](#set-up-constants)
    * [Set Up Helper Functions for Normalization](#set-up-helper-functions-for-normalization)
    * [Normalize Across All Experiment Sets](#normalize-across-all-experiment-sets)
* [Calculate Fold Changes](#calculate-fold-changes)
    * [Calculate Fold Change and Log2 Fold Change Between Timepoints](#calculate-fold-change-and-log-fold-change-between-timepoints)
    * TODO: [Calculate Fold Change and Log2 Fold Change Against Plasmid](#calculate-fold-change-and-log-fold-change-against-plasmid)
* [Calculate Fitness and Pi Scores](#calculate-fitness-and-pi-scores)   
* [Calculate Averages and P-Values for Gene Pair Metrics](#calculate-averages-and-p-values-for-gene-pair-metrics)
* [Outstanding Questions](#outstanding-questions)
    * [Aggregated Normalization](#aggregated-normalization)
    * [Non-Targeting Control Collapse](#non-targeting-control-collapse)
    * [Plasmid Fold-Changes](#plasmid-fold-changes)
    * [Fitness Score P-Values](#fitness-score-p-values)

<a name = "review-goals"></a>

## Review goals

The Workflow.pdf document provided by Dongxin Zhao on 02/11/2016 defines Module 2 this way:

*Module 2: raw counts -> fitness and GI score* 

* Output normalized counts for each raw counts file
* Output the fold change data 
    * Fold change VS d3: d14/d3 and d28/d3
    * Fold change VS plasmid: d3/plasmid, d14/plasmid and d28/plasmid
* Output fitness and its P-value for each time point
    * Use Boutros’ method (Nature Method, 2011)
* Output the GI π score and its P-value for each time point and sort

On 03/17/2016, Dongxin also provided a methods.docx document that includes details on many of the above steps; information from this document will be included below with the affected step.

[Table of Contents](#table-of-contents)

<a name="set-up-environment"></a>

## Set Up Environment

<a name="create-virtual-environment-and-install-software"></a>

### Create Virtual Environment and Install Software

In a terminal window, run
<pre>
cd Documents/Projects/Projects2016/mali_crispr/
mkvirtualenv -p /usr/local/bin/python3 mali_cripsr_venv
pip install --upgrade pip
pip3 install pandas
pip3 install scipy
pip3 install jupyter
jupyter notebook
</pre>

Then open this notebook.

[Table of Contents](#table-of-contents)

<a name="initialize-shared-variables-and-functions"></a>

### Initialize Shared Variables and Functions

In [6]:
# below is 80-character ruler to ensure code lines don't get too long
"123456789012345678901234567890123456789012345678901234567890123456789012345678"
g_mod1_output_dir = ("/Users/Birmingham/Projects/20160210_mali_crispr/real_data")
g_mod1_output_suffix = "_raw.csv"

[Table of Contents](#table-of-contents)

<a name="normalize-raw-counts"></a>


## Normalize Raw Counts

methods.docx specifies that normalization should be performed by the following approach:
![](images/2016-03-18_9.48.48.png)

I contacted Dongxin for clarification on several points.  He wrote back:

>(1) Each day is a separate experiment. For example, i = "ADA_chr20_43280245__ABL1_chr9_133589790" and j = day 3. [snip /]

>(2) Each construct must be separate. For example, "ADA_chr20_43280245__ABL1_chr9_133589790" and "ADA_chr20_43280245__ABL1_chr9_133738283" are two different pairs because the two ABL gRNAs target to different positions. And "ADA_chr20_43280245__ABL1_chr9_133589790" is different from "ABL1_chr9_133589790__ADA_chr20_43257696", and different from "
ABL1_chr9_133589790__NonTargetingControlGuideForHuman0352", too.

>(3) According to eqn.1, sj = medium (xij / x'i), I think the normalized count yij = xij / sj,  instead of yij = xij * sj.

He then clarified point 1:

>For our real data, we will have two replicates. Actually they are two independent experiment sets, which means experiment A(day3, day14, day28) and B(day3, day14, day28) but NOT the replicates at each time point like day3 (A,B), day14(A,B) and day28(A,B). Therefore, please always process each replicate independently. 

>For the question 1, i = gRNA pair and j = time point (day3, day14, day28). So x’i should be the geometric mean of the experiments (day3, day14 and day28) with gRNA pair i.

In addition, Dr. Mali clarified point 2:

>You are right -- position in the pairing does not matter: GeneA-GeneB or GeneB-GeneA do indeed refer to the same gene pair and there will be 9 such combinations total and hence are processed as a group.

but

>Just to clarify, in the original designs, I don't believe there is any pairing of the same gRNA-pair in both orientations. We did pair genes in both orientations, but never the same identical gRNA-pair.


The input to the normalization will be the output of Module 1, which according to the specifications will include at least "gRNA pair ID, targeted gene pair, raw counts".  

[Table of Contents](#table-of-contents)

<a name="set-up-constants"></a>

### Set Up Constants

In [19]:
class data_file_format:
    construct_id_header = "construct_id"
    pair_separator = "__"
    piece_separator = "_"
    position_header_base = "gene_position"
    position1_header = "gene_position1"
    position2_header = "gene_position2"
    gene_pair_header = "gene_pair"
    num_expected_genes = 2
    input_headers = [construct_id_header, "expt1_day3", "expt2_day3", 
                     "expt1_day14", "expt2_day14", 
                     "expt1_day28", "expt2_day28"]
    general_headers = [construct_id_header, gene_pair_header, 
                      position1_header, position2_header]
    experiment_set_prefixes = ["expt1", "expt2"]
    # per discussion with Roman 03/22/2016, I expect every raw count file will 
    # include the same number of gRNA pair ids--the whole library.  
    # Any gRNA pair id that was not sequenced will have a zero count.
    expected_num_constructs = 24908
    earliest_timept = "day3"    
    negative_control_genes = ['NonTargetingControlGuideForHuman']
    all_expts_prefix = "all_expts"
    by_gene_pair_suffix = "_by_gene_pair.csv"
    
    def get_headers_for_experiment_set(column_names, experiment_set_prefix, 
                                       include_enum):
        result = []
        if include_enum == 0:
            pass
        elif include_enum == 1:
            result = [x for x in data_file_format.general_headers]
        elif include_enum == 2:
            result = [x for x in data_file_format.general_headers 
                      if x != data_file_format.construct_id_header]
        # end if
        expt_specific_headers = [x for x in column_names 
                                 if x.startswith(experiment_set_prefix)]
        result.extend(expt_specific_headers)
        return result
    
    
    def get_headers_for_expt_timepoints(column_names, timepoint_suffix, 
                                        find_all_other_timepoints, 
                                        experiment_set_prefix):
        timept_headers_for_expt_set = data_file_format.get_headers_for_experiment_set(
            column_names, experiment_set_prefix, 0)
        if find_all_other_timepoints:
            result = [x for x in timept_headers_for_expt_set 
                      if not x.endswith(timepoint_suffix)]
        else:
            result = [x for x in timept_headers_for_expt_set 
                      if x.endswith(timepoint_suffix)]
        return result
 
        
    def get_dataframe_for_expt(all_expts_dataframe, experiment_set_prefix, 
                                       include_general_enum):
        copied_df = all_expts_dataframe.copy()
        column_names = copied_df.columns.values
        all_relevant_headers = data_file_format.get_headers_for_experiment_set(
            column_names, experiment_set_prefix, include_general_enum)
        data_frame = copied_df[all_relevant_headers]
        return data_frame
    

def add_series_to_dataframe(dataframe, series, header):
    dataframe.loc[:, header] = pandas.Series(series, 
        index=dataframe.index)        
    

# raw count column names
g_grna_pair_id_header = "gRNA_pair"
g_raw_counts_header = "raw_counts"
g_geo_mean_header = "geo_mean_across_expts"

# new column header/file text
g_size_factor_txt = "_size_factor"
g_norm_counts_txt = "_norm_counts"
g_fold_change_txt = "_fc"
g_log2_fold_change_txt = "_log2fc"
g_log2_fitness_score_txt = "_log2fitness"
g_log2_pi_score_txt = "_log2pi_score"
g_norm_suffix = "_norm.csv"
g_log2_fold_change_suffix = "_log2_fc.csv"
g_pi_suffix = "_pi_scores.csv"

[Table of Contents](#table-of-contents)

<a name="set-up-helper-functions-for-normalization"></a>

### Set Up Helper Functions for Normalization

In [8]:
from functools import partial
import glob
import pandas
import re

def get_file_path(file_type, expt_name):
    return os.path.join(g_mod1_output_dir, "{0}{1}".format(expt_name, file_type))   


get_norm_file_path = partial(get_file_path, g_norm_suffix)
get_log2fc_file_path = partial(get_file_path, g_log2_fold_change_suffix)


def get_norm_col_header(curr_day_header):
    return "{0}{1}".format(curr_day_header,g_norm_counts_txt)


def get_filepaths_from_wildcard(directory, wildcard_filename):
    wildpath = os.path.join(directory, wildcard_filename)
    return [x for x in glob.glob(wildpath)]    

[Table of Contents](#table-of-contents)

<a name="normalize-across-all-experiment-sets"></a>

### Normalize Across All Experiment Sets

Normalized counts are calculated per construct.

In [9]:
import numpy
import os
import pandas
import scipy.stats.mstats

    
def load_all_expts(directory, raw_counts_filename, col_headers):
    filepath = os.path.join(directory, raw_counts_filename) 
    all_expts_dataframe = pandas.read_table(filepath, names=col_headers, 
                                            skiprows=1)
    return all_expts_dataframe


def expand_info_from_construct_id(construct_row, file_format):
    construct_id = construct_row[file_format.construct_id_header]
    construct_id_pieces = construct_id.split(file_format.pair_separator)
    
    if len(construct_id_pieces) != file_format.num_expected_genes:
        raise ValueError(("Construct {0} split into {1} pieces"
                         "rather than {2}").format(construct_id, 
                         len(construct_id_pieces), 
                         file_format.num_expected_genes))
        
    results = []
    for index in range(0, file_format.num_expected_genes):
        gene_pieces = construct_id_pieces[index].split(
            file_format.piece_separator)
        gene = gene_pieces[0]
        
        # TODO: Double-check with customer if this is correct:
        # if the "gene" name is for one of the non-targeting 
        # controls, compress to the control's name
        for control_gene in file_format.negative_control_genes:
            if gene.startswith(control_gene):
                gene = control_gene
                
        results.append(gene)
    
    # Ensure that gene pairs are treated as combinations not
    # permutations (order not important) by sorting gene list
    # alphabetically
    results.sort()
    
    gene_pair = file_format.pair_separator.join(results)
    results.append(gene_pair)
    return tuple(results)


def expand_frame_with_gene_info(dataframe, file_format):    
    row_expansion_func = lambda x: expand_info_from_construct_id(
        x, file_format)
    expanded_tuples_df = dataframe.apply(row_expansion_func, axis=1) 
    (dataframe[file_format.position1_header], 
     dataframe[file_format.position2_header], 
     dataframe[file_format.gene_pair_header]) = zip(*expanded_tuples_df)
    

def normalize_across_all_expts(all_expts_dataframe, file_format):
    output_path = get_norm_file_path(file_format.all_expts_prefix)
    days = [x for x in file_format.input_headers if x != file_format.construct_id_header]

    # calculate geometric mean across all days for each 
    # gRNA pair id and add as new column
    raw_counts_across_days = all_expts_dataframe[days]
    # NB: 1 = apply to each row
    geo_means = raw_counts_across_days.apply(
        scipy.stats.mstats.gmean, axis=1)
    add_series_to_dataframe(all_expts_dataframe, geo_means, g_geo_mean_header)

    # for each day
    for curr_day_header in days:
        # calculate raw count value / geometric mean for all 
        # gRNA pair ids in day, then remove infinite and NaN values
        raw_div_by_geo_mean = (all_expts_dataframe[curr_day_header] / 
                               all_expts_dataframe[g_geo_mean_header])   
        temp = raw_div_by_geo_mean.replace([numpy.inf, -numpy.inf], 
                                           numpy.nan)
        raw_div_by_geo_mean = temp.dropna()

        # calculate size factor for day as median of above
        # NB: 0 = apply to each column
        size_factor = raw_div_by_geo_mean.median(axis=0)             

        # add new column holding size factor for day 
        # (same for all gRNA pair ids)         
        size_factor_vector = [size_factor for x in range(
                0, file_format.expected_num_constructs)]
        size_factor_header = "{0}{1}".format(curr_day_header, 
                                             g_size_factor_txt)
        add_series_to_dataframe(all_expts_dataframe, size_factor_vector, 
                                size_factor_header)

        # vector-calculate normalized count value for all gRNA pair ids 
        # in this day as raw count value for gRNA pair id in this day 
        # divided by size factor for day, add as new column
        norm_counts = (all_expts_dataframe[curr_day_header] / 
                       all_expts_dataframe[size_factor_header])
        norm_header = get_norm_col_header(curr_day_header)
        add_series_to_dataframe(all_expts_dataframe, norm_counts, 
                                norm_header)        

        all_expts_dataframe.to_csv(output_path, index=False)
        

def normalize_all_expts(directory, raw_counts_filename):
    expts_dataframe = load_all_expts(directory, raw_counts_filename, 
                                     data_file_format.input_headers)
    expand_frame_with_gene_info(expts_dataframe, data_file_format)
    normalize_across_all_expts(expts_dataframe, data_file_format)
    
normalize_all_expts(g_mod1_output_dir, "prashant_all_counts.txt")

[Table of Contents](#table-of-contents)

<a name="calculate-fold-changes"></a>

## Calculate Fold Changes

methods.docx specifies that fold changes should be calculated by the following approach:
![](images/2016-03-21_11.50.44.png)

I also received the following clarifications from Dr. Mali on two minor methods questions:

1. The methods specify "These fold changes are log-transformed”; do you want log base 2 or log base 10 transformation?  The former is more common for biological screening applications.
    *  log base 2 is preferred.
2. Do you want the average of the log fold changes for the constructs in the gene pair, or the log of the average fold change for constructs in a gene pair?  Again, I would anticipate the former (see http://www.experts-exchange.com/questions/22667100/log-transformation-of-average-or-average-of-log-transformation.html for relevant reasoning).
    * average of the log fold changes for the constructs in the gene pair is preferred.

[Table of Contents](#table-of-contents)

<a name="calculate-fold-change-and-log-fold-change-between-timepoints"></a>

### Calculate Fold Change and Log2 Fold Change Between Timepoints

These values are calculated per construct.

In [10]:
import numpy
import os
import pandas

def input_plus_one(x):
    return x+1


def get_norm_plus_one_series(data_frame, source_token):
    # get normalized count + 1 for input source (a timept or plasmid)
    norm_source_token = get_norm_col_header(source_token)
    return data_frame[norm_source_token].apply(input_plus_one)

    
def add_fold_change_series(dataframe, numerator_series, numerator_name, 
                           denominator_series, denominator_name):
    fold_change = (numerator_series / denominator_series)
    fold_change_header = "{0}_vs_{1}{2}".format(numerator_name, 
                                                
                                                
        denominator_name, g_fold_change_txt)        
    add_series_to_dataframe(dataframe, fold_change, fold_change_header)
    return fold_change, fold_change_header
  
    
def add_log2fc_series(dataframe, fc_series, fc_header):
    log2_fold_change = fc_series.apply(numpy.log2)   
    log2_fold_change_header = fc_header.replace(g_fold_change_txt, 
        g_log2_fold_change_txt)
    add_series_to_dataframe(dataframe, log2_fold_change, 
        log2_fold_change_header)   
    
    
def add_fold_change_and_log2fc(dataframe, numerator_series, numerator_name, 
                           denominator_series, denominator_name):
    fc_series, fc_header = add_fold_change_series(dataframe, 
        numerator_series, numerator_name, denominator_series, denominator_name)
    add_log2fc_series(dataframe, fc_series, fc_header)
    
    
def calculate_fold_changes_across_days(file_format):
    norm_csv_path = get_norm_file_path(file_format.all_expts_prefix)   
    output_path = get_log2fc_file_path(file_format.all_expts_prefix)
    all_expts_dataframe = pandas.read_csv(norm_csv_path)
    columns = file_format.input_headers
    
    for curr_expt_set_prefix in file_format.experiment_set_prefixes:
        earliest_timept_header = file_format.get_headers_for_expt_timepoints(
            columns, file_format.earliest_timept, False, curr_expt_set_prefix)[0]
        later_timept_headers = file_format.get_headers_for_expt_timepoints(
            columns, file_format.earliest_timept, True, curr_expt_set_prefix)
        earliest_timept_norm_plus_one = get_norm_plus_one_series(
            all_expts_dataframe, earliest_timept_header)

        for curr_later_timept in later_timept_headers:
            # calculate fold change and log2 fold change for each gRNA pair, 
            # and add to data frame
            later_timept_norm_plus_one = get_norm_plus_one_series(
                all_expts_dataframe, curr_later_timept)
            add_fold_change_and_log2fc(all_expts_dataframe, 
                later_timept_norm_plus_one, curr_later_timept, 
                earliest_timept_norm_plus_one, earliest_timept_header)

    all_expts_dataframe.to_csv(output_path, index=False)
        
calculate_fold_changes_across_days(data_file_format)

[Table of Contents](#table-of-contents)

<a name="calculate-fold-change-and-log-fold-change-against-plasmid"></a>

### Calculate Fold Change and Log2 Fold Change Against Plasmid

The workflow.pdf requests "Fold change VS plasmid: d3/plasmid, d14/plasmid and d28/plasmid".  As above, these are calculated per construct.

In [11]:
# Roman opines that if experiment day values are being compared to plasmid values, 
# then plasmid values must be included in the normalization step.
# I want to ensure the customer understand this before modifying the normalization
# code.

def calculate_fold_changes_against_plasmid(file_format):
    all_expts_dataframe = pandas.Dataframe([])  # TODO: GET REAL NORMALIZED DAY DATA
    plasmid_norm_counts = pandas.Series([])  # TODO: GET REAL NORMALIZED? PLASMID DATA
    plasmid_norm_plus_one = get_norm_plus_one_series(data_frame, source_token)
    
    # TODO: Find out how customer wants plasmid fold changes to be output--same file
    # as other fold-changes?  Different?  What downstream use planned?
    # This will affect how I grab the normalized day data (e.g., grouped by expt or not)
    
# TODO: fill out plasmid fold change calculations

[Table of Contents](#table-of-contents)

<a name="calculate-fitness-and-pi-scores"></a>

## Calculate Fitness and Pi Scores

Fitness scores are calculated per gene.  Pi scores are calculated per construct.  

In [12]:
from functools import partial
import os
import pandas

def get_relevant_col_names(col_name_suffix, data_frame):
    col_names = data_frame.columns.values    
    relevant_col_names = [x for x in col_names if x.endswith(
            col_name_suffix)]
    return relevant_col_names


get_fc_col_names = partial(get_relevant_col_names, g_log2_fold_change_txt)
get_pi_col_names = partial(get_relevant_col_names, g_log2_pi_score_txt)


def get_indexes_for_id(id, dataframe, list_of_col_headers):
    filters = [dataframe[x] == id for x in list_of_col_headers]
    indexes = numpy.any(filters, axis=0)
    return indexes


def get_data_for_gene(curr_gene, dataframe, data_col_name, position_cols_list):
    indexes_for_gene = get_indexes_for_id(curr_gene, dataframe, 
                                          position_cols_list)
    dataframe_for_curr_gene = dataframe[indexes_for_gene] 
    return dataframe_for_curr_gene[data_col_name]


def get_median_for_gene(gene_name, dataframe, data_col_name, position_cols_list):
    data_for_gene = get_data_for_gene(gene_name, dataframe, data_col_name, 
                                      position_cols_list)
    return data_for_gene.median()
        
    
def get_half_medians_for_gene_col(gene_series, dataframe, data_col_name, 
                                  position_cols_list):
    custom_median_func = lambda x: get_median_for_gene(x, dataframe, 
                                                       data_col_name, 
                                                       position_cols_list)
    medians_by_gene = gene_series.apply(custom_median_func) 
    medians_by_gene_series = pandas.Series(medians_by_gene, 
                                           index=gene_series.index)
    return 0.5 * medians_by_gene_series


def get_unique_vals_in_column(dataframe, col_name):
    uniques_array = dataframe[col_name].unique()
    uniques_series = pandas.Series(uniques_array, index=uniques_array)    
    return uniques_series
    
    
def compute_residual(row, data_col_name, other_position_header, 
                     genes_main_effects):
    raw_data = row[data_col_name]
    other_gene = row[other_position_header]
    other_main_effect = genes_main_effects[other_gene]
    residual = raw_data - other_main_effect
    return residual
    
    
def compute_residuals(dataframe, data_col_name, other_position_header, 
                      other_genes_main_effects):
    custom_residuals_func = lambda x: compute_residual(x, data_col_name, 
                                                       other_position_header, 
                                                       other_genes_main_effects)
    # NB: 1 = apply to each row
    residuals = dataframe.apply(custom_residuals_func, axis=1)  
    return residuals


def get_residuals_median_for_gene(gene_name, residuals_series, index_frame, 
                                  focus_position_header, main_effects):
    indexes_for_gene = get_indexes_for_id(gene_name, index_frame, 
                                          [focus_position_header])
    residuals_for_gene = residuals_series[indexes_for_gene]
    median_residual = residuals_for_gene.median()
    main_effects[gene_name] = median_residual
    

def iterate_main_effects(dataframe, data_col_name, focus_position_header, 
                         other_position_header, main_effects):
    residuals = compute_residuals(dataframe, data_col_name, 
                                  other_position_header, main_effects)
    unique_genes_in_focus_pos = get_unique_vals_in_column(dataframe, 
                                                          focus_position_header)
    custom_residuals_median_func = lambda x: get_residuals_median_for_gene(x, 
        residuals, dataframe, focus_position_header, main_effects)
    unique_genes_in_focus_pos.apply(custom_residuals_median_func)   
    
    
def get_base_main_effects(unique_genes_series, data_frame, data_col_header, 
                          position1_header, position2_header):
    main_effects_by_gene = get_half_medians_for_gene_col(unique_genes_series, 
        data_frame, data_col_header, [position1_header, position2_header])
    
    for _ in range(0,20):
        iterate_main_effects(data_frame, data_col_header, position1_header, 
                             position2_header, main_effects_by_gene)
        iterate_main_effects(data_frame, data_col_header, position2_header, 
                             position1_header, main_effects_by_gene)
    
    return main_effects_by_gene


def get_control_mean(unique_genes_in_focus_pos, negative_control_genes, 
                     main_effects):
    present_neg_controls = unique_genes_in_focus_pos[
        unique_genes_in_focus_pos.isin(negative_control_genes)]
    if len(present_neg_controls) < 1:    
        raise ValueError("No negative controls found in among genes {0}"
                         .format(unique_genes_in_focus_pos))
    main_effects_for_present_negs = main_effects[present_neg_controls]
    return main_effects_for_present_negs.mean()

    
def normalize_main_effect(gene_index, control_mean, main_effects):
    base_main_effect = main_effects[gene_index]
    normalized_main_effect = base_main_effect - control_mean
    main_effects[gene_index] = normalized_main_effect
    

def subtract_control_mean_from_main_effects(dataframe, focus_position_header, 
                                            neg_control_genes, main_effects):
    unique_genes_in_focus_pos = get_unique_vals_in_column(dataframe, 
                                                          focus_position_header)
    control_mean = get_control_mean(unique_genes_in_focus_pos, neg_control_genes, 
                                    main_effects)
    custom_normalize_func = lambda x: normalize_main_effect(x, control_mean, 
                                                            main_effects)
    unique_genes_in_focus_pos.apply(custom_normalize_func)  
    return control_mean


def calculate_pi_score(row, main_effects, sum_control_means, data_col_header, 
                       position1_header, position2_header):
    position1_gene_id = row[position1_header]
    position2_gene_id = row[position2_header]
    real_data = row[data_col_header]
    
    # model = main effect for gene in position 1 + 
    # main effect for gene in position 2 + sum of neg means    
    model_prediction = (main_effects[position1_gene_id] 
                        + main_effects[position2_gene_id] + sum_control_means)
    pi_score = real_data - model_prediction
    return pi_score


def calculate_pi_scores_for_expt(dataframe, main_effects, sum_control_means, 
                                 data_col_header, position1_header, 
                                 position2_header):
    custom_pi_score_func = lambda x: calculate_pi_score(x, main_effects, 
        sum_control_means,data_col_header, position1_header, position2_header)
    # NB: 1 = rows
    pi_scores = dataframe.apply(custom_pi_score_func, axis=1)
    return pi_scores
    
    
def calculate_pi_scores(file_format):
    log2fc_csv_path = get_log2fc_file_path(file_format.all_expts_prefix)     
    pi_output_path = get_file_path(g_pi_suffix, file_format.all_expts_prefix)
    fitness_output_path = get_file_path("_fitness_scores.csv", 
                                        file_format.all_expts_prefix)
    all_expts_dataframe = pandas.read_csv(log2fc_csv_path)
    
    vals_in_position_cols = all_expts_dataframe[[file_format.position1_header, 
        file_format.position2_header]].values.ravel()
    unique_genes = pandas.Series(vals_in_position_cols).unique()
    unique_genes_series = pandas.Series(unique_genes, index=unique_genes)
    fitness_scores_df = unique_genes_series.to_frame(name="gene")
    
    for curr_expt_set_prefix in file_format.experiment_set_prefixes:
        data_frame = file_format.get_dataframe_for_expt(all_expts_dataframe, 
            curr_expt_set_prefix, 2)
        
        data_col_headers =  get_fc_col_names(data_frame)   
        for data_col_header in data_col_headers:
            main_effects = get_base_main_effects(unique_genes_series, 
                data_frame, data_col_header, file_format.position1_header, 
                file_format.position2_header)           
            
            # test values: [4,11,1062,1068]
            negative_control_genes = file_format.negative_control_genes
            position1_control_mean = subtract_control_mean_from_main_effects(
                data_frame, file_format.position1_header, 
                negative_control_genes, main_effects)
            position2_control_mean = subtract_control_mean_from_main_effects(
                data_frame, file_format.position2_header, 
                negative_control_genes, main_effects)
            
            fitness_score_header = data_col_header.replace(g_log2_fold_change_txt,
                g_log2_fitness_score_txt)
            add_series_to_dataframe(fitness_scores_df, main_effects, 
                                    fitness_score_header)              
            
            sum_control_means = position1_control_mean + position2_control_mean

            pi_scores_for_expt = calculate_pi_scores_for_expt(data_frame, 
                                        main_effects, sum_control_means, 
                                        data_col_header, file_format.position1_header,  
                                        file_format.position2_header)
            
            pi_score_header = data_col_header.replace(g_log2_fold_change_txt,
                g_log2_pi_score_txt)            
            add_series_to_dataframe(all_expts_dataframe, pi_scores_for_expt, 
                                    pi_score_header)
            
    all_expts_dataframe.to_csv(pi_output_path, index=False)
    
    # NB: pi(ab) = f(ab) - f(a) - f(b) - controls
    # where f values are log2-transformed fitness scores
    # Thus, the gene-pair "fitness scores" are actually just the contents
    # of the <expt>_avg_log2fc_by_gene_pair file
    # And the fitness scores of the individual genes are the main effects
    fitness_scores_df.to_csv(fitness_output_path, index=False)
        
calculate_pi_scores(data_file_format)

[Table of Contents](#table-of-contents)

<a name="calculate-averages-and-p-values-for-gene-pair-metrics"></a>

### Calculate Averages and P-Values for Gene Pair Metrics

These values are calculated per gene pair.

In [18]:
from scipy import stats

def get_p_vals_from_tuples(t_and_p_tuple_series):
    custom_extract_func = lambda x: x[1]
    # NB: 1 = apply to each row
    p_vals_series = t_and_p_tuple_series.apply(custom_extract_func)  
    return p_vals_series


def calculate_means_and_ps(all_expts_dataframe, file_format):
    relevant_col_functions = [get_fc_col_names, get_pi_col_names]
    calculated_series = []
    
    for curr_expt_set_prefix in file_format.experiment_set_prefixes:
        data_frame = file_format.get_dataframe_for_expt(all_expts_dataframe, 
            curr_expt_set_prefix, 1)
        grouped_by_gene_pair = data_frame.groupby([file_format.gene_pair_header, 
                                                   file_format.position1_header, 
                                                   file_format.position2_header])

        for relevant_cols_func in relevant_col_functions:
            relevant_col_names = relevant_cols_func(data_frame)
            for curr_col in relevant_col_names:
                col_vals_for_groups = grouped_by_gene_pair[curr_col]
                stats_test_func = lambda x: stats.ttest_1samp(x, 0)

                t_stat_and_p_tuples = col_vals_for_groups.aggregate(
                    stats_test_func)
                new_p_series = get_p_vals_from_tuples(t_stat_and_p_tuples)
                new_p_series.name =  curr_col + "_p"
                calculated_series.append(new_p_series)

                new_mean_series = col_vals_for_groups.mean()
                new_mean_series.name =  curr_col + "_mean"
                calculated_series.append(new_mean_series)

    new_data_frame = pandas.concat(calculated_series, axis=1)
    return new_data_frame


def calculate_log2fc_and_pi_mean_and_p(file_format):
    pi_scores_path = get_file_path(g_pi_suffix, 
                                   file_format.all_expts_prefix)  
    all_expts_dataframe = pandas.read_csv(pi_scores_path)
    output_path = get_file_path(file_format.by_gene_pair_suffix, 
                                file_format.all_expts_prefix)

    new_data_frame = calculate_means_and_ps(all_expts_dataframe, 
                                            file_format)
    new_data_frame.to_csv(output_path, header=True, 
                          index_label=[file_format.gene_pair_header,
                          file_format.position1_header, 
                          file_format.position2_header])        
        
calculate_log2fc_and_pi_mean_and_p(data_file_format)

[Table of Contents](#table-of-contents)

<a name = "outstanding-questions"></a>
    
## Outstanding Questions

<a name = "aggregated-normalization"></a>

### Aggregated Normalization

The customer had indicated data will be generated several different times; we need to know more about the source and use of these different data groupings.  If they will be treated independently, then they can each be normalized (and subsequently analyzed) separately.  However, if they will be compared, then Roman considers that they should all be normalized together.  This seems at odds with the customer's goal of analyzing the data as it becomes available.

[Table of Contents](#table-of-contents)

<a name = "non-targeting-control-collapse"></a>

### Non-Targeting Control Collapse

Double-check with customer if this is correct: if the "gene" name is for one of the non-targeting controls, compress to the control's name.

[Table of Contents](#table-of-contents)

### Plasmid Fold Changes

<a name = "plasmid-fold-changes"></a>

Roman opines that if experiment day values are being compared to plasmid values, then plasmid values must be included in the normalization step.  I want to ensure the customer understand this before modifying the normalization
code.

Also: how does customer wants plasmid fold changes to be output--same file as other fold-changes?  Different?  What downstream use planned? This will affect how I grab the normalized day data (e.g., grouped by expt or not).

[Table of Contents](#table-of-contents)
    
<a name = "fitness-score-p-values"></a>    
    
### Fitness Score P-Values

Need clarification from customer on how they want p-values calculated for 
fitness scores. Same way as for pi scores: 1-sample t-test on ... what
distribution?  Combination (median, rather than mean) is done inside
get_residuals_median_for_gene, which is ITERATED 20 times, although
the number of constructs containing the gene is the same every time ... 
if we did t-test at this stage, would use residuals_for_gene as 
distribution to test as different from zero?  Do we just test after final
iteration?  What about fact that the base main effects (from the 
median of residuals) have the control means *subtracted* from them later 
(long after combination to single effect per gene)?

Or ... is it possible they want p-values on fitness scores of gene *pairs*,
directly analogous to p-values on pi values of gene pairs?  That's easy,
so implemented it just in case that's what they want.

[Table of Contents](#table-of-contents)